In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
data = pd.read_csv("t20_data_clean.zip", compression="zip")

In [23]:
data.sample(3)

,match_id,dates,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner
509802,1383075,2023-07-23,"Goldenacre, Edinburgh",Jersey,field,Jersey,Italy,2,32,2,22,8.000000,184.0,9.306122,Italy
591134,1424761,2024-03-20,"Achimota Senior Secondary School B Field, Accra",Ghana,bat,Ghana,Kenya,1,2,0,5,2.000000,NaN,NaN,Kenya
468878,1273745,2021-11-04,Dubai International Cricket Stadium,Australia,field,Bangladesh,Australia,1,70,8,85,4.666667,NaN,NaN,Australia


In [24]:
data["dates"] = pd.to_datetime(data["dates"], errors="coerce")
data["year"] = data["dates"].dt.year

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018006 entries, 0 to 1018005
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   match_id           1018006 non-null  int64         
 1   dates              1018006 non-null  datetime64[ns]
 2   venue              1018006 non-null  object        
 3   toss_winner        1018006 non-null  object        
 4   toss_decision      1018006 non-null  object        
 5   batting_team       1018006 non-null  object        
 6   bowling_team       1018006 non-null  object        
 7   innings            1018006 non-null  int64         
 8   runs_so_far        1018006 non-null  int64         
 9   wickets_so_far     1018006 non-null  int64         
 10  balls_faced        1018006 non-null  int64         
 11  run_rate           1018006 non-null  float64       
 12  target_runs        468804 non-null   float64       
 13  required_run_rate  467509 n

In [26]:
data["batting_team_wins"] = (data["winner"] == data["batting_team"]).astype(int)
data.sample(3)

,match_id,dates,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,year,batting_team_wins
985679,1172918,2019-02-21,Asian Institute of Technology Ground,China,field,Kuwait,China,1,17,3,34,2.756757,NaN,NaN,China,2019,0
96449,1153698,2019-02-10,Seddon Park,India,field,New Zealand,India,1,144,2,85,9.600000,NaN,NaN,New Zealand,2019,1
887868,1381221,2023-08-13,Central Broward Regional Park Stadium Turf Gro...,India,bat,West Indies,India,2,132,2,90,8.800000,166.0,6.8,West Indies,2023,1


In [27]:
# Fill missing numeric with -1
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].fillna(-1)
data.drop(columns=["dates"], inplace=True)

In [28]:
data.sample(5)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,year,batting_team_wins
400368,1199503,ICC Academy,Papua New Guinea,field,Bermuda,Papua New Guinea,1,11,1,10,5.500000,-1.0,-1.0,Papua New Guinea,2019,0
51472,1375852,Perth Stadium,West Indies,bat,West Indies,Australia,1,189,5,111,9.947368,-1.0,-1.0,West Indies,2024,1
912928,1321294,Kerava National Cricket Ground,Luxembourg,bat,Luxembourg,Bulgaria,1,32,1,29,6.193548,-1.0,-1.0,Luxembourg,2022,1
188000,1173055,Manuka Oval,West Indies,bat,West Indies,Pakistan,1,124,7,120,6.200000,-1.0,-1.0,Pakistan,2020,0
217193,1150538,Providence Stadium,Ireland,bat,Ireland,Australia,1,19,0,24,4.750000,-1.0,-1.0,Australia,2018,0


In [29]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in ['venue', 'toss_winner', 'toss_decision', 'batting_team', 'bowling_team']:
    le = LabelEncoder()
    
    # Fit on all unique values in the FULL dataset (df), not just training split
    all_values = data[col].dropna().unique().tolist()
    le.fit(all_values)
    
    # Transform the full column now
    data[col] = le.transform(data[col])
    
    # Save encoder
    label_encoders[col] = le


In [30]:
data.sample(3)

,match_id,venue,toss_winner,toss_decision,batting_team,bowling_team,innings,runs_so_far,wickets_so_far,balls_faced,run_rate,target_runs,required_run_rate,winner,year,batting_team_wins
912285,1418541,223,48,0,19,49,2,73,9,95,4.515464,259.0,44.64,Japan,2024,0
141758,963697,203,89,1,41,90,1,45,3,43,5.510204,-1.0,-1.00,Sri Lanka,2016,0
931731,1286982,153,38,0,38,52,1,81,1,65,7.363636,-1.0,-1.00,Hong Kong,2021,1


In [31]:
data_inn1 = data[data["innings"] == 1].copy()
data_inn2 = data[data["innings"] == 2].copy()

# Features for innings 1 → drop target-related
features_inn1 = [col for col in data_inn1.columns if col not in [
    "match_id", "winner", "batting_team_wins", 
    "target_runs", "runs_required", "required_rr"
]]

# Features for innings 2 → keep all
features_inn2 = [col for col in data_inn2.columns if col not in [
    "match_id", "winner", "batting_team_wins"
]]

In [32]:
from sklearn.model_selection import GroupShuffleSplit

# Split so same match doesn't leak between train/test
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx1, test_idx1 = next(gss1.split(data_inn1[features_inn1], data_inn1["batting_team_wins"], groups=data_inn1["match_id"]))

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx2, test_idx2 = next(gss2.split(data_inn2[features_inn2], data_inn2["batting_team_wins"], groups=data_inn2["match_id"]))

X1_train, X1_test = data_inn1[features_inn1].iloc[train_idx1], data_inn1[features_inn1].iloc[test_idx1]
y1_train, y1_test = data_inn1["batting_team_wins"].iloc[train_idx1], data_inn1["batting_team_wins"].iloc[test_idx1]

X2_train, X2_test = data_inn2[features_inn2].iloc[train_idx2], data_inn2[features_inn2].iloc[test_idx2]
y2_train, y2_test = data_inn2["batting_team_wins"].iloc[train_idx2], data_inn2["batting_team_wins"].iloc[test_idx2]

print("Train size:", X1_train.shape, "Test size:", X1_test.shape)
print("Train size:", X2_train.shape, "Test size:", X2_test.shape)

Train size: (439135, 12) Test size: (109610, 12)
Train size: (374039, 13) Test size: (94765, 13)


In [33]:
from lightgbm import LGBMClassifier

# First innings model
model_inn1 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1200,
    random_state=42
)

model_inn1.fit(
    X1_train, y1_train,
    eval_set=[(X1_test, y1_test)],
    eval_metric="auc",
)


[LightGBM] [Info] Number of positive: 218554, number of negative: 220581
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1217
[LightGBM] [Info] Number of data points in the train set: 439135, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497692 -> initscore=-0.009232
[LightGBM] [Info] Start training from score -0.009232


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.002
,n_estimators,1200
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [34]:
# Second innings model
model_inn2 = LGBMClassifier(
    objective="binary",
    learning_rate=0.002,
    num_leaves=31,
    n_estimators=1000,
    random_state=42
)

model_inn2.fit(
    X2_train, y2_train,
    eval_set=[(X2_test, y2_test)],
    eval_metric="auc",
)

[LightGBM] [Info] Number of positive: 169417, number of negative: 204622
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003383 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1709
[LightGBM] [Info] Number of data points in the train set: 374039, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.452939 -> initscore=-0.188801
[LightGBM] [Info] Start training from score -0.188801


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.002
,n_estimators,1000
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [35]:
from sklearn.metrics import roc_auc_score, accuracy_score

# First innings
y1_pred_proba = model_inn1.predict_proba(X1_test)[:, 1]
y1_pred = (y1_pred_proba >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred))

# Second innings
y2_pred_proba = model_inn2.predict_proba(X2_test)[:, 1]
y2_pred = (y2_pred_proba >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred))

First Innings AUC: 0.7408066718397872
First Innings Accuracy: 0.6755952924003285
Second Innings AUC: 0.9123204032039482
Second Innings Accuracy: 0.8279428058882499


In [36]:
import pickle

model_bundle = {
    "model_inn1": model_inn1,
    "model_inn2": model_inn2,
    "label_encoders": label_encoders,
    "features_inn1": features_inn1,
    "features_inn2": features_inn2
}

with open("pipe.pkl", "wb") as f:
    pickle.dump(model_bundle, f)

print("✅ Saved two-model setup to pipe.pkl")


✅ Saved two-model setup to pipe.pkl


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid_logistic = {
    'C': [0.01, 0.1, 1],       # Regularization strength
    'penalty': ['l2'],            # Norm used in the penalization
    'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}
inn1_model_logistic = LogisticRegression(max_iter=1000)
inn1_grid_search_logistic = GridSearchCV(inn1_model_logistic, param_grid_logistic, cv=3, scoring='roc_auc', n_jobs=-1)
inn1_grid_search_logistic.fit(X1_train, y1_train)

inn2_model_logistic = LogisticRegression(max_iter=1000)
inn2_grid_search_logistic = GridSearchCV(inn2_model_logistic, param_grid_logistic, cv=3, scoring='roc_auc', n_jobs=-1)
inn2_grid_search_logistic.fit(X2_train, y2_train)

print("Best Parameters (inning 1):", inn1_grid_search_logistic.best_params_)
print("Best Cross-validation Accuracy (inning 1):", inn1_grid_search_logistic.best_score_)

print("Best Parameters (inning 2):", inn2_grid_search_logistic.best_params_)
print("Best Cross-validation Accuracy (inning 2):", inn2_grid_search_logistic.best_score_)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:425: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/optimi

Best Parameters (inning 1): {'C': 0.01, 'penalty': 'l2', 'solver': 'sag'}
Best Cross-validation Accuracy (inning 1): 0.7328448976686911
Best Parameters (inning 2): {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Cross-validation Accuracy (inning 2): 0.9111782693816367


In [ ]:
# First innings
y1_pred_proba_logistic = inn1_grid_search_logistic.predict_proba(X1_test)[:, 1]
y1_pred_logistic = (y1_pred_proba_logistic >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba_logistic))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred_logistic))

# Best Parameters (inning 1): {'C': 0.01, 'penalty': 'l2', 'solver': 'sag'}

# Second innings
y2_pred_proba_logistic = inn2_grid_search_logistic.predict_proba(X2_test)[:, 1]
y2_pred_logistic = (y2_pred_proba_logistic >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba_logistic))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred_logistic))

# Best Parameters (inning 2): {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

First Innings AUC: 0.7216775825908632
First Innings Accuracy: 0.6606514004196697
Second Innings AUC: 0.9113077265293417
Second Innings Accuracy: 0.8232153221125943


In [ ]:
import xgboost as xgb

param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

inn1_model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
inn1_grid_search_xgb = GridSearchCV(inn1_model_xgb, param_grid_xgb, cv=3, scoring='roc_auc', n_jobs=-1)
inn1_grid_search_xgb.fit(X1_train, y1_train)

inn2_model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
inn2_grid_search_xgb = GridSearchCV(inn2_model_xgb, param_grid_xgb, cv=3, scoring='roc_auc', n_jobs=-1)
inn2_grid_search_xgb.fit(X2_train, y2_train)

print("Best Parameters (inning 1):", inn1_grid_search_xgb.best_params_)
print("Best Cross-validation Accuracy (inning 1):", inn1_grid_search_xgb.best_score_)

print("Best Parameters (inning 2):", inn2_grid_search_xgb.best_params_)
print("Best Cross-validation Accuracy (inning 2):", inn2_grid_search_xgb.best_score_)

In [52]:
# First innings
y1_pred_proba_xgb = inn1_grid_search_xgb.predict_proba(X1_test)[:, 1]
y1_pred_xgb = (y1_pred_proba_xgb >= 0.5).astype(int)
print("First Innings AUC:", roc_auc_score(y1_test, y1_pred_proba_xgb))
print("First Innings Accuracy:", accuracy_score(y1_test, y1_pred_xgb))

# Best Parameters (inning 1): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.7}

# Second innings
y2_pred_proba_xgb = inn2_grid_search_xgb.predict_proba(X2_test)[:, 1]
y2_pred_xgb = (y2_pred_proba_xgb >= 0.5).astype(int)
print("Second Innings AUC:", roc_auc_score(y2_test, y2_pred_proba_xgb))
print("Second Innings Accuracy:", accuracy_score(y2_test, y2_pred_xgb))

# Best Parameters (inning 2): {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.7}

First Innings AUC: 0.7623782905767983
First Innings Accuracy: 0.6896086123528875
Second Innings AUC: 0.9144360739700117
Second Innings Accuracy: 0.8263177333403683
